In [1]:
from time import time
start = time()

In [2]:
import os
GPUs = ','.join([str(i) for i in range(0,8)])
num_gpus = len(GPUs.split(','))
os.environ['CUDA_VISIBLE_DEVICES'] = GPUs
os.environ["RAPIDS_NO_INITIALIZE"] = "1" #Vjawa: Added

In [3]:
import dask_cudf
from dask.distributed import Client
from dask_cuda import LocalCUDACluster
import xgboost as xgb
import gc

def enable_spilling():
    import cudf
    cudf.set_option("spill", True)

def get_cluster():
    cluster = LocalCUDACluster()
    client = Client(cluster)
    return client


In [4]:

client = get_cluster()
client

2023-06-21 17:31:17,425 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space-10093/worker-jo2kgt4g', purging
2023-06-21 17:31:17,425 - distributed.preloading - INFO - Creating preload: dask_cuda.initialize
2023-06-21 17:31:17,425 - distributed.preloading - INFO - Import preload module: dask_cuda.initialize
2023-06-21 17:31:17,425 - distributed.preloading - INFO - Creating preload: dask_cuda.initialize
2023-06-21 17:31:17,425 - distributed.preloading - INFO - Creating preload: dask_cuda.initialize
2023-06-21 17:31:17,426 - distributed.preloading - INFO - Import preload module: dask_cuda.initialize
2023-06-21 17:31:17,426 - distributed.preloading - INFO - Import preload module: dask_cuda.initialize
2023-06-21 17:31:17,426 - distributed.preloading - INFO - Creating preload: dask_cuda.initialize
2023-06-21 17:31:17,426 - distributed.preloading - INFO - Import preload module: dask_cuda.initialize
2023-06-21 17:31:17,426 - distributed.preloading - INFO

Connection method: Cluster object,Cluster type: dask_cuda.LocalCUDACluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 8
Total threads: 8,Total memory: 1.48 TiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:43239,Workers: 8
Dashboard: http://127.0.0.1:8787/status,Total threads: 8
Started: Just now,Total memory: 1.48 TiB
Comm: tcp://127.0.0.1:37033,Total threads: 1
Dashboard: http://127.0.0.1:40695/status,Memory: 188.82 GiB
Nanny: tcp://127.0.0.1:35953,


In [5]:
%%time

users = dask_cudf.read_parquet('/raid/otto/Otto-Comp/pqs/train_v152_*.pq').persist()#,split_row_groups=True)
users.head()

CPU times: user 2.08 s, sys: 1.48 s, total: 3.56 s
Wall time: 11 s


,user,item,pos,wgt,clicks,score_1,score_3,score_4,score_5,score_6,...,cart_repeat9,buy_ratio10,count_item10,unique_item10,hour_mean10,day_mean10,repeat10,buy_ratio_std10,hour_std10,day_std10
0,11098528,11830,0,-1.000,0,-1.0,-1.0,-1.0,-1.0,-1.0,...,1.965084,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
1,11098528,588923,1,14.500,0,2.0,1.0,-3.0,2.0,4.0,...,2.031830,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
2,11098528,884502,2,12.485,0,1.0,1.0,-3.0,1.0,2.0,...,1.925403,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
3,11098528,1182614,3,12.470,0,1.0,1.0,-3.0,1.0,1.5,...,1.904306,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
4,11098528,571762,4,11.990,0,1.0,1.0,-3.0,1.0,2.0,...,1.953917,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>


In [6]:
%%time
users.shape[0].compute()

CPU times: user 1.83 s, sys: 1.65 s, total: 3.48 s
Wall time: 26.3 s


180125100

In [7]:
%%time
users.memory_usage().sum().compute()/2**30

CPU times: user 239 ms, sys: 36.1 ms, total: 275 ms
Wall time: 390 ms


105.77028216049075

In [8]:
VER = 186
USE = 'clicks'

In [9]:
FEATURES = users.columns[2:]
TARS = [USE]
FEATURES = [f for f in FEATURES if f not in TARS]
print(len(FEATURES))
print( FEATURES)
print(TARS)

152
['pos', 'wgt', 'score_1', 'score_3', 'score_4', 'score_5', 'score_6', 'score_7', 'score_10', 'score_11', 'score_12', 'score_13', 'score_30', 'score_14', 'score_15', 'score_16', 'score_17', 'score_18', 'score_19', 'score_20', 'score_21', 'score_22', 'score_23', 'score_24', 'score_25', 'score_26', 'buy_ratio_5', 'count_item_5', 'count_user_5', 'repeat_5', 'buy_ratio_std_5', 'item_day_std_5', 'user_day_std_5', 'buy_ratio_0_6', 'buy_ratio_1_6', 'buy_ratio_2_6', 'buy_ratio_3_6', 'buy_ratio_4_6', 'buy_ratio_5_6', 'buy_ratio_6_6', 'count_item_0_6', 'count_item_1_6', 'count_item_2_6', 'count_item_3_6', 'count_item_4_6', 'count_item_5_6', 'count_item_6_6', 'count_user_0_6', 'count_user_1_6', 'count_user_2_6', 'count_user_3_6', 'count_user_4_6', 'count_user_5_6', 'count_user_6_6', 'item_day_std_6', 'user_day_std_6', 'buy_ratio', 'count_item', 'count_user', 'hour_mean', 'day_mean', 'repeat', 'buy_ratio_std', 'hour_std', 'day_std', 'buy_ratio3', 'count_item3', 'count_user3', 'hour_mean3', 'day

In [10]:
%%time

dtrain = xgb.dask.DaskQuantileDMatrix(client, users[FEATURES], users['clicks'])

CPU times: user 2.23 s, sys: 1.04 s, total: 3.27 s
Wall time: 27.2 s


In [11]:
FOLDS = 5
SEED = 42

LR = 0.1

# XGB MODEL PARAMETERS
xgb_parms = { 
    'max_depth':4, 
    'learning_rate':LR, 
    'subsample':0.7,
    'colsample_bytree':0.5, 
    'eval_metric':'map',
    'objective':'binary:logistic',
    'scale_pos_weight':8,
    'tree_method':'gpu_hist',
    'predictor':'gpu_predictor',
    'random_state':SEED
}

In [12]:
%%time

output = xgb.dask.train(
        client,
        xgb_parms,
        dtrain,
        num_boost_round=100,
        evals=[(dtrain, "train")],
    )

INFO:distributed.scheduler:Receive client connection: Client-worker-483b51ac-1094-11ee-a6b1-5cff35c18da6
INFO:distributed.core:Starting established connection to tcp://127.0.0.1:38274
INFO:distributed.scheduler:Receive client connection: Client-worker-486ca10b-1094-11ee-a6b7-5cff35c18da6
INFO:distributed.core:Starting established connection to tcp://127.0.0.1:38280
INFO:distributed.scheduler:Receive client connection: Client-worker-48ffd5f8-1094-11ee-a6ad-5cff35c18da6
INFO:distributed.core:Starting established connection to tcp://127.0.0.1:38282
INFO:distributed.scheduler:Receive client connection: Client-worker-492a8196-1094-11ee-a6a8-5cff35c18da6
INFO:distributed.core:Starting established connection to tcp://127.0.0.1:38298
INFO:distributed.scheduler:Receive client connection: Client-worker-49604a77-1094-11ee-a6aa-5cff35c18da6
INFO:distributed.core:Starting established connection to tcp://127.0.0.1:38308
INFO:distributed.scheduler:Receive client connection: Client-worker-496226f0-109

[0]	train-map:0.16833
[1]	train-map:0.17518
[2]	train-map:0.17642
[3]	train-map:0.17726
[4]	train-map:0.18079
[5]	train-map:0.18160
[6]	train-map:0.18084
[7]	train-map:0.18096
[8]	train-map:0.18111
[9]	train-map:0.18112
[10]	train-map:0.18574
[11]	train-map:0.18563
[12]	train-map:0.18629
[13]	train-map:0.18641
[14]	train-map:0.18645
[15]	train-map:0.18738
[16]	train-map:0.18748
[17]	train-map:0.18756
[18]	train-map:0.18784
[19]	train-map:0.18802
[20]	train-map:0.18901
[21]	train-map:0.19048
[22]	train-map:0.19108
[23]	train-map:0.19129
[24]	train-map:0.19131
[25]	train-map:0.19149
[26]	train-map:0.19149
[27]	train-map:0.19149
[28]	train-map:0.19168
[29]	train-map:0.19195
[30]	train-map:0.19254
[31]	train-map:0.19268
[32]	train-map:0.19315
[33]	train-map:0.19351
[34]	train-map:0.19406
[35]	train-map:0.19495
[36]	train-map:0.19490
[37]	train-map:0.19542
[38]	train-map:0.19596
[39]	train-map:0.19619
[40]	train-map:0.19626
[41]	train-map:0.19646
[42]	train-map:0.19653
[43]	train-map:0.1966

In [13]:
duration = time()-start
duration

135.20442724227905